# Projeto 1 - Ciência dos Dados

Nome: Mateus Marinheiro

Nome: Henrique Albuquerque


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Mateus Marinhero\Documents\Insper\2023\2o Semestre\Ciência dos Dados\Projeto1\Projeto-1-C-Dados


Carregando a base de dados com os tweets classificados manualmente:

In [6]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target
0,Fique atento na compra. Ainda está com problem...,Relevante
1,Talvez a tradução antiga tivesse um pouco mais...,Relevante
2,"Sinceramente, também estou tentando entender c...",Relevante
3,Escolhi o livro por acreditar no tema e indica...,Irrelevante
4,"Recebi o produto avariado, no mesmo dia reclam...",Relevante


In [7]:
train.Target.value_counts()

Irrelevante    153
Relevante      147
Name: Target, dtype: int64

In [8]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,"Não sou de desistir de livro, mas neste não te...",Irrelevante
1,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Relevante
2,Tao ruim quanto o primeiro. Fonte não justific...,Relevante
3,"Sou fã do canal, estava na primeira sessão da ...",Irrelevante
4,Péssimo. O autor claramente tem um visão disto...,Irrelevante


In [9]:
test.Target.value_counts()

Irrelevante    162
Relevante       38
Name: Target, dtype: int64

___
## Classificador automático de feedbacks de produtos da Amazon


Será criado um classificador segundo a teoria de probabilidade de Naïve Bayes, classificando notícias atuais sobre o mercado digital como:

- Relevante: Mensagens consideradas como relevantes serão aqueles que trazem alguma crítica à direcionadas a Amazon, sejam elas, em relação à preços, envios e outros problemas. Também serão considerados relevantes críticas feitas às editoras, sendo assim, mensagens criticando a tradução, formatação do texto, qualidade do produto entre outros.</li>
    
- Não Relevante: Mensagens consideradas como não relevantes serão aquelas que trazem qualquer tipo de opinião sobre o livro, qualquer tipo de elogio e qualquer outra coisa que não se enquadra como relevante.  </li>
    

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### 1 - Função _cleanup_

Função que irá retirar das mensagens recebidas todas as pontuações.

In [10]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

#### 2 - Filtrando os Feedbacks

In [46]:
palavras_relevantes = []             #Lista com as palvras de feedbacks relevantes.
palavras_nao_relevantes = []         #Lista com as palvras de feedbacks não relevantes.

train_relevantes = train.loc[train.Target == 'Relevante', :]
train_nao_relevantes = train.loc[train.Target == 'Não Relevante', :]

for i in range(len(train_relevantes)):
    palavras_relevantes += cleanup(train_relevantes.iloc[i, 0].lower()).split()

for i in range(len(train_nao_relevantes)):
    palavras_nao_relevantes += cleanup(train_nao_relevantes.iloc[i, 0].lower()).split()

#Tirando númreros das listas de palavras.

for i in range(len(palavras_relevantes)):
    for caractere in palavras_relevantes[i]:
        if caractere.isdigit():
            palavras_relevantes[i] = ''
    
for i in range(len(palavras_nao_relevantes)):
    for caractere in palavras_nao_relevantes[i]:
        if caractere.isdigit():
            palavras_nao_relevantes[i] = ''



#### 3 - Criando pd.series para cada Target

In [44]:
serie_relevantes = 

False

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**